<a href="https://colab.research.google.com/github/Gayeon6423/2023BigContest/blob/main/%5B%EC%83%81%EA%B6%8C%2C%EC%8B%A0%EC%9A%A9%2C%EB%8C%80%ED%9A%8C%5D_DATA_MERGE_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import random
import os

In [4]:
# 구글 드라이브 임포트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 시드 세팅
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# 경로 설정
basic_file_path = '/content/drive/MyDrive/프로젝트/2023 빅콘테스트/data/'
market_2022_file_path = '/content/drive/MyDrive/프로젝트/2023 빅콘테스트/data/상권_2022/'
market_2023_file_path = '/content/drive/MyDrive/프로젝트/2023 빅콘테스트/data/상권_2023/'
contest_file_path = '/content/drive/MyDrive/프로젝트/2023 빅콘테스트/data/contest data/'
mrege_file_path = '/content/drive/MyDrive/프로젝트/2023 빅콘테스트/data/MERGE_DATA/'
credict_file_path = '/content/drive/MyDrive/프로젝트/2023 빅콘테스트/data/DESILO_PLATFORM_DATA/'

### 상권 데이터: 계절성 지수 기반 월 확장, 행정동 코드 추가

In [ ]:
# 데이터 불러오기
## 상권 데이터
MARKET_2022_GROUPED = pd.read_csv('/content/MARKET_2022_GROUPED_EXPAND.csv', encoding = 'cp949')
MARKET_2023_GROUPED = pd.read_csv('/content/MARKET_2023_GROUPED_EXPAND.csv', encoding = 'cp949')

## 신용 데이터
KCD_2022_2023 = pd.read_csv('/content/SOHO_MERGE_DATA_LIST.csv')

## 대회 데이터
contest_2022 = pd.read_csv('/content/소상공인 매출등급 예상 대상 필지.csv', encoding = 'cp949')
contest_2023 = pd.read_csv('/content/필지단위 소상공인 매출등급 정보.csv', encoding = 'cp949')

# 상권코드-행정동코드 매핑 데이터
market_loc_map = pd.read_csv('/content/market_loc_map.csv', encoding = 'cp949')

In [ ]:
# 분기별 상권데이터를 소호 신용데이터를 활용하여 월별로 확장(계절성 지수)
## 기준_분기_코드 칼럼의 값을 변경하여 월 칼럼 생성
MARKET_2022_GROUPED['월'] = MARKET_2022_GROUPED['기준_분기_코드'].apply(lambda x: '1 2 3' if x == 1 else ('4 5 6' if x == 2 else ('7 8 9' if x == 3 else ('10 11 12' if x == 4 else x))))
MARKET_2023_GROUPED['월'] = MARKET_2023_GROUPED['기준_분기_코드'].apply(lambda x:'1 2 3' if x == 1 else x)

## '월' 칼럼의 값을 공백을 기준으로 분리
MARKET_2022_GROUPED['월'] = MARKET_2022_GROUPED['월'].str.split()
MARKET_2023_GROUPED['월'] = MARKET_2023_GROUPED['월'].str.split()

## '월' 칼럼을 분리하기 위해 데이터프레임을 재구성
tmp_2022 = pd.DataFrame({'기준_월_코드': np.concatenate(MARKET_2022_GROUPED['월'].values)})
tmp_2023 = pd.DataFrame({'기준_월_코드': np.concatenate(MARKET_2023_GROUPED['월'].values)})

## 데이터를 인덱스 기준으로 확장
expand_mon_2022 = MARKET_2022_GROUPED.loc[MARKET_2022_GROUPED.index.repeat(3)].reset_index(drop=True)
expand_mon_2022 = expand_mon_2022.drop(columns = '월')
expand_mon_2023 = MARKET_2023_GROUPED.loc[MARKET_2023_GROUPED.index.repeat(3)].reset_index(drop=True)
expand_mon_2023 = expand_mon_2023.drop(columns = '월')

## 확장된 데이터 병합
market_2022_mon = pd.concat([expand_mon_2022,tmp_2022],axis=1)
market_2023_mon = pd.concat([expand_mon_2023,tmp_2023],axis=1)

## DATA_CRTR_YM 칼럼 생성(기준년도+기준월)
market_2022_mon['기준_월_코드'] = market_2022_mon['기준_월_코드'].apply(lambda x : '0' + x if len(x) == 1 else x)
market_2023_mon['기준_월_코드'] = market_2023_mon['기준_월_코드'].apply(lambda x : '0' + x if len(x) == 1 else x)

#market_2022_mon.iloc[:,0] = market_2022_mon['기준_년_코드'].astype('str') + market_2022_mon['기준_월_코드'].astype('str')
market_2022_mon = market_2022_mon.rename(columns = {'기준_년_코드':'기준일자'})
market_2022_mon['기준일자'] = market_2022_mon['기준일자'].astype('int64')

# market_2023_mon.iloc[:,0] = market_2023_mon['기준_년_코드'].astype('str') + market_2023_mon['기준_월_코드'].astype('str')
market_2023_mon = market_2023_mon.rename(columns = {'기준_년_코드':'기준일자'})
market_2023_mon['기준일자'] = market_2023_mon['기준일자'].astype('int64')

In [ ]:
## 신용데이터
KCD_상권코드 = KCD_2022_2023["상권코드"].unique().tolist()
KCD_COLUMNS = KCD_2022_2023.columns.tolist()
for column_name in ['Unnamed: 0','기준일자','상권_식별자','상권코드','상권명','외식업종_중분류','소상공인연령_평균','소상공인업력_평균']:
    KCD_COLUMNS.remove(column_name)

## 계절성 지수 계산
SEASONAL_PATTERN = KCD_2022_2023.groupby(["기준일자"])[KCD_COLUMNS].mean()
SEASONAL_PATTERN = SEASONAL_PATTERN / SEASONAL_PATTERN.mean()
pd.set_option("display.max_column", None)
SEASONAL_PATTERN = SEASONAL_PATTERN.reset_index()

## 계절성 지수 평균 값 계산
SEASONAL_PATTERN_mean = SEASONAL_PATTERN[SEASONAL_PATTERN.iloc[:,1:].columns].mean(axis=1)
SEASONAL_PATTERN['SEASEONAL_MEAN'] = SEASONAL_PATTERN_mean
SEASONAL_PATTERN = SEASONAL_PATTERN[['기준일자','SEASEONAL_MEAN']]
tmp_2022 = pd.merge(market_2022_mon, SEASONAL_PATTERN, on = ['기준일자'],how='left')
tmp_2023 = pd.merge(market_2023_mon, SEASONAL_PATTERN, on = ['기준일자'],how='left')

## 상권데이터 숫자형 칼럼에 계절성 지수 반영
num_col_2022 = tmp_2022.select_dtypes(include='number').columns.tolist()
num_col_2022 = [i for i in num_col_2022 if i not in ('기준일자','SEASEONAL_MEAN','기준_분기_코드','상권_코드','소득_구간_코드')]
num_col_2023 = tmp_2023.select_dtypes(include='number').columns.tolist()
num_col_2023 = [i for i in num_col_2023 if i not in ('기준일자','SEASEONAL_MEAN','기준_분기_코드','상권_코드','소득_구간_코드')]

tmp_2022[num_col_2022] = tmp_2022[num_col_2022].mul(tmp_2022['SEASEONAL_MEAN_x'],axis=0)
MARKET_2022_GROUPED_EXPAND = tmp_2022
tmp_2023[num_col_2023] = tmp_2023[num_col_2023].mul(tmp_2023['SEASEONAL_MEAN_x'],axis=0)
MARKET_2023_GROUPED_EXPAND = tmp_2023

In [ ]:
# 상권코드, 행정동 코드 매핑
## 칼럼명 전처리
market_loc_map_2022 = market_loc_map[['상권_코드_2022','행정동_코드','법정동_코드','STDG_EMD_CD']]
market_loc_map_2022 = market_loc_map_2022.rename(columns = {'상권_코드_2022':'상권_코드'})

market_loc_map_2023 = market_loc_map[['상권_코드_2023','행정동_코드','법정동_코드','STDG_EMD_CD']]
market_loc_map_2023 = market_loc_map_2023.rename(columns = {'상권_코드_2023':'상권_코드'})

## 상권 데이터, 행정동 데이터 병합
MARKET_2022_GROUPED_EXPAND = pd.merge(MARKET_2022_GROUPED_EXPAND, market_loc_map_2022,on = ['상권_코드'], how='left')
MARKET_2023_GROUPED_EXPAND = pd.merge(MARKET_2023_GROUPED_EXPAND, market_loc_map_2023, on = ['상권_코드'], how='left')

## 중복 칼럼 제거
MARKET_2022_GROUPED_EXPAND = MARKET_2022_GROUPED_EXPAND.loc[:,~MARKET_2022_GROUPED_EXPAND.columns.duplicated()]
MARKET_2023_GROUPED_EXPAND = MARKET_2023_GROUPED_EXPAND.loc[:,~MARKET_2023_GROUPED_EXPAND.columns.duplicated()]
MARKET_2022_GROUPED_EXPAND.drop(['행정동_코드_y','법정동_코드_y','STDG_EMD_CD_y'],axis=1)
MARKET_2023_GROUPED_EXPAND.drop(['행정동_코드_y','법정동_코드_y','STDG_EMD_CD_y'],axis=1)
MARKET_2022_GROUPED_EXPAND = MARKET_2022_GROUPED_EXPAND.rename(columns = {'행정동_코드_x':'행정동_코드','법정동_코드_x':'법정동_코드','STDG_EMD_CD_x':'STDG_EMD_CD'})
MARKET_2023_GROUPED_EXPAND = MARKET_2023_GROUPED_EXPAND.rename(columns = {'행정동_코드_x':'행정동_코드','법정동_코드_x':'법정동_코드','STDG_EMD_CD_x':'STDG_EMD_CD'})

## 중복 행 제거
MARKET_2022_GROUPED_EXPAND.drop_duplicates(subset=['기준일자', '행정동_코드', '법정동_코드', 'STDG_EMD_CD'], keep='first', inplace=True)
MARKET_2023_GROUPED_EXPAND.drop_duplicates(subset=['기준일자', '행정동_코드', '법정동_코드', 'STDG_EMD_CD'], keep='first', inplace=True)

## 데이터 csv파일로 저장
MARKET_2022_GROUPED_EXPAND.to_csv('MARKET_2022_GROUPED_EXPAND2.csv',encoding='euc-kr',index=False)
MARKET_2023_GROUPED_EXPAND.to_csv('MARKET_2023_GROUPED_EXPAND2.csv',encoding='euc-kr',index=False)

In [ ]:
MARKET_2022_GROUPED_EXPAND2 = pd.read_csv('/content/MARKET_2022_GROUPED_EXPAND2.csv', encoding = 'cp949')
MARKET_2023_GROUPED_EXPAND2 = pd.read_csv('/content/MARKET_2023_GROUPED_EXPAND2.csv', encoding = 'cp949')

### 신용 데이터 : 행정동 코드 추가

In [ ]:
# 상권코드, 행정동 코드 매핑
KCD_2022_2023['기준_년도_코드'] = KCD_2022_2023['기준일자'].astype('str').str[:4]
credit_2022 = KCD_2022_2023[KCD_2022_2023['기준_년도_코드'] == '2022']
credit_2023 = KCD_2022_2023[KCD_2022_2023['기준_년도_코드'] == '2023']

## 칼럼명 전처리
credit_2022 = credit_2022.rename(columns = {'상권코드':'상권_코드'})
credit_2023 = credit_2023.rename(columns = {'상권코드':'상권_코드'})

## 상권 데이터, 행정동 데이터 병합
credit_2022 = pd.merge(credit_2022, market_loc_map_2022, on = ['상권_코드'], how='inner')
credit_2023 = pd.merge(credit_2023, market_loc_map_2023, on = ['상권_코드'], how='inner')

## 중복 칼럼 제거
credit_2022 = credit_2022.loc[:,~credit_2022.columns.duplicated()]
credit_2023 = credit_2023.loc[:,~credit_2023.columns.duplicated()]

## 중복 행 제거
# credit_2022.drop_duplicates(subset=['행정동_코드', '법정동_코드', 'STDG_EMD_CD'], keep='first', inplace=True)
# credit_2023.drop_duplicates(subset=['행정동_코드', '법정동_코드', 'STDG_EMD_CD'], keep='first', inplace=True)
credit_2022 = credit_2022.iloc[:,1:].rename(columns = {'기준일자':'DATA_CRTR_YM'})
credit_2023 = credit_2023.iloc[:,1:].rename(columns = {'기준일자':'DATA_CRTR_YM'})

# ## 데이터 csv파일로 저장
# market_2022.to_csv('market_2022.csv',encoding='euc-kr',index=False)
# market_2023.to_csv('market_2023.csv',encoding='euc-kr',index=False)

In [ ]:
credit_2023

,DATA_CRTR_YM,상권_식별자,상권_코드,상권명,외식업종_중분류,소호사업장_수,경영위기사업장_총합,프랜차이즈사업장_총합,소상공인연령_평균,소상공인업력_평균,정규고용인원_평균,사업장방문고객수_평균,사업장방문신규고객수_평균,매출액_변동계수,카드매출액_변동계수,배달매출액_변동계수,주말카드매출액_변동계수,주말배달매출액_변동계수,매입액_변동계수,매출총이익_변동계수,부가가치세_변동계수,부가세차감전영업이익_변동계수,사업장방문고객수_변동계수,사업장방문신규고객수_변동계수,사업장임대면적_변동계수,월임대료_변동계수,임대보증금_변동계수,손익분기점매출액_변동계수,객단가_변동계수,신규고객단가_변동계수,기준_년도_코드,행정동_코드,법정동_코드,STDG_EMD_CD


In [ ]:
len(KCD_2022_2023)

2692

In [ ]:
# 상권 데이터, 대회 데이터 병합(기준 날짜, 상권 코드 기준)

## 문자형 -> 숫자형
market_2022['DATA_CRTR_YM'] = market_2022['DATA_CRTR_YM'].astype('int64')
market_2023['DATA_CRTR_YM'] = market_2023['DATA_CRTR_YM'].astype('int64')

## 상권 데이터, 대회 데이터 병합
market_contest_2022 = pd.merge(market_2022, contest_2022, on = ['DATA_CRTR_YM','STDG_EMD_CD'], how='right')
market_contest_2023 = pd.merge(market_2023, contest_2023, on = ['STDG_EMD_CD'], how='right')

## 중복 칼럼 제거
market_contest_2022 = market_contest_2022.loc[:,~market_contest_2022.columns.duplicated()]
market_contest_2023 = market_contest_2023.loc[:,~market_contest_2023.columns.duplicated()]

## 중복 행 제거
market_contest_2022.drop_duplicates(subset=['DATA_CRTR_YM','LT_UNQ_NO','INDUSTRY_CD','STDG_EMD_CD'], keep='first', inplace=True)
market_contest_2023.drop_duplicates(subset=['LT_UNQ_NO','INDUSTRY_CD','STDG_EMD_CD'], keep='first', inplace=True)

## 고유 id 생성
market_contest_2022['id'] = market_contest_2022['DATA_CRTR_YM'].astype('str') + market_contest_2022['LT_UNQ_NO'].astype('str') + market_contest_2022['STDG_EMD_CD'].astype('str') + market_contest_2022['INDUSTRY_CD'].astype('str')
market_contest_2023['id'] = market_contest_2023['LT_UNQ_NO'].astype('str') + market_contest_2023['STDG_EMD_CD'].astype('str') + market_contest_2023['INDUSTRY_CD'].astype('str')

## 데이터 csv파일로 저장
market_contest_2022.to_csv('market_contest_2022.csv',encoding='euc-kr',index=False)
market_contest_2023.to_csv('market_contest_2023.csv',encoding='euc-kr',index=False)

NameError: ignored

In [ ]:
## 상권, 신용, 대회 데이터 병합
# total_df_2022 = pd.merge(credit_df,market_contest_2022,right_index=True, left_index=True, how='inner')
# total_df_2023 = pd.merge(credit_df,market_contest_2023,right_index=True, left_index=True, how='outer')
total_df_2022 = pd.merge(credit_df,market_contest_2022,on = ['DATA_CRTR_YM','상권_코드'], how='inner')
total_df_2023 = pd.merge(credit_df,market_contest_2023,on = ['상권_코드'], how='right')

## 중복 칼럼 제거
total_df_2022 = total_df_2022.loc[:,~total_df_2022.columns.duplicated()]
total_df_2023 = total_df_2023.loc[:,~total_df_2023.columns.duplicated()]

## 중복 행 제거
total_df_2022.drop_duplicates(subset=['id'], keep='first', inplace=True)
total_df_2023.drop_duplicates(subset=['id'], keep='first', inplace=True)

## 데이터 csv파일로 저장
# total_df_2022.to_csv('total_df_2022.csv',encoding='euc-kr',index=False)
# total_df_2023.to_csv('total_df_2023.csv',encoding='euc-kr',index=False)